In [13]:
import pandas as pd
import yfinance as yf
import numpy as np

# Fetch and Clean Data


In [14]:
# companies = ["POLYCAB", "ITC", "LTTS", "HCLTECH", "NMDC", "DRREDDY", "DEEPAKNTR"]
companies = [
    "HDFCAMC",
    "CUMMINSIND",
    "MCDOWELL-N",
    "LTTS",
    "HCLTECH",
    "DRREDDY",
]
N = len(companies)
rf = 0.0721


def get_returns(companies: list[str]):
    companies_str = " ".join([c + ".NS" for c in companies])
    df: pd.DataFrame = yf.download(companies_str, period="max").dropna()["Adj Close"]
    return df.pct_change().dropna()


def get_latest_price(companies: list[str]):
    companies_str: str = " ".join([c + ".NS" for c in companies])
    df: pd.DataFrame = yf.download(companies_str, period="max").dropna()["Adj Close"]
    return df


price = get_latest_price(companies)
df = get_returns(companies)
df

[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed


,CUMMINSIND.NS,DRREDDY.NS,HCLTECH.NS,HDFCAMC.NS,LTTS.NS,MCDOWELL-N.NS
Date,,,,,,
2018-08-07,-0.019391,0.010920,0.012859,-0.025331,-0.003745,-0.022370
2018-08-08,-0.004028,-0.005888,-0.004387,0.005480,-0.000702,0.029347
2018-08-09,-0.039635,-0.003474,-0.008190,-0.017194,0.005765,0.008429
2018-08-10,0.000536,-0.010457,0.009774,-0.000086,0.039149,-0.007785
2018-08-13,0.027015,0.012329,0.017598,-0.024787,0.019126,0.003964
...,...,...,...,...,...,...
2024-01-04,0.010523,-0.015122,-0.012380,0.027523,0.022385,0.009470
2024-01-05,0.002860,-0.001121,0.009050,-0.006839,-0.002266,-0.006810
2024-01-08,0.002952,-0.012732,0.010539,0.014091,-0.011317,-0.002952


In [15]:
exp_ret = (((1 + df.mean()) ** 252) - 1).to_numpy()

exp_ret = exp_ret.reshape(len(exp_ret), 1)
exp_ret

array([[0.31870142],
       [0.24797284],
       [0.32272884],
       [0.20548085],
       [0.39205593],
       [0.17994564]])

In [16]:
cov_mat = (df.cov() * 252).to_numpy()
cov_mat

array([[0.10483888, 0.00708157, 0.01441232, 0.02792478, 0.02359838,
        0.02475433],
       [0.00708157, 0.06866456, 0.01734311, 0.01558069, 0.0178763 ,
        0.01954789],
       [0.01441232, 0.01734311, 0.07786287, 0.02318099, 0.04347692,
        0.01999048],
       [0.02792478, 0.01558069, 0.02318099, 0.10441392, 0.03171838,
        0.03452324],
       [0.02359838, 0.0178763 , 0.04347692, 0.03171838, 0.13915715,
        0.02601364],
       [0.02475433, 0.01954789, 0.01999048, 0.03452324, 0.02601364,
        0.09631282]])

In [17]:
def sharpe(w: np.ndarray, mu: np.ndarray, sigma: np.ndarray, rf: float):
    return ((w.T @ mu - rf) / (w.T @ sigma @ w) ** 0.5)[0][0]


def init_w(n: int, random=True):
    if random:
        w = np.random.uniform(0, 1, (n, 1))
        w /= w.sum()
        return w
    else:
        w = np.zeros((n, 1))
        w[0][0] = 1
        return w


sharpe(init_w(N), exp_ret, cov_mat, rf)

1.1909058366293208

# Genetic Algorithm


In [18]:
def fitness(w: np.ndarray, tangency=True):
    if abs(w.sum() - 1) > 1e-2:
        return -(1e9 + 7)
    if tangency:
        return sharpe(w, exp_ret, cov_mat, 0.05)
    else:
        return (-w.T @ cov_mat @ w).flatten()[0]

## Hyper-Parameters


In [19]:
n_iter = 2000
mutation_variation = 0.1
offsprings = 200
top_select = offsprings // 2

## Run the Model


In [53]:
solutions = np.array([init_w(N) for _ in range(offsprings)])

for i in range(n_iter):
    fitness_values = np.array([fitness(w, tangency=False) for w in solutions])
    ranked_indices = np.argsort(fitness_values)[::-1]
    solutions = solutions[ranked_indices.astype(int)]

    best = solutions[:top_select]

    selected_indices = np.random.choice(top_select, size=offsprings)
    mutated = best[selected_indices] * np.random.uniform(
        1 - mutation_variation / 2, 1 + mutation_variation / 2, (offsprings, N, 1)
    )
    mutated /= mutated.sum(axis=1, keepdims=True)

    solutions = mutated

for i in range(10):
    print(
        f"optimal solution {i+1}: sharpe ratio = {round(sharpe(solutions[i], exp_ret, cov_mat, rf), 3)} with weights = {np.round(solutions[i].T, 4).flatten().tolist()}"
    )

w = (
    pd.DataFrame(solutions.reshape(solutions.shape[0], solutions.shape[1]))
    .mean()
    .to_numpy()
)
w /= w.sum()

w = w.reshape(w.shape[0], 1)

optimal solution 1: sharpe ratio = 1.105 with weights = [0.1761, 0.3325, 0.2221, 0.0991, 0.0412, 0.129]
optimal solution 2: sharpe ratio = 1.107 with weights = [0.1821, 0.3306, 0.2176, 0.1048, 0.0416, 0.1233]
optimal solution 3: sharpe ratio = 1.106 with weights = [0.1841, 0.333, 0.208, 0.1128, 0.0445, 0.1177]
optimal solution 4: sharpe ratio = 1.113 with weights = [0.1845, 0.3275, 0.2246, 0.1033, 0.0416, 0.1186]
optimal solution 5: sharpe ratio = 1.098 with weights = [0.1927, 0.3129, 0.2097, 0.1147, 0.037, 0.133]
optimal solution 6: sharpe ratio = 1.108 with weights = [0.1889, 0.3373, 0.2102, 0.101, 0.0419, 0.1207]
optimal solution 7: sharpe ratio = 1.104 with weights = [0.1845, 0.3168, 0.223, 0.1122, 0.0378, 0.1258]
optimal solution 8: sharpe ratio = 1.106 with weights = [0.1837, 0.3273, 0.2219, 0.0962, 0.0395, 0.1314]
optimal solution 9: sharpe ratio = 1.108 with weights = [0.1983, 0.3293, 0.2015, 0.1046, 0.0433, 0.123]
optimal solution 10: sharpe ratio = 1.111 with weights = [0.189

# Overall Portfolio


In [60]:
capital = 100000

print(f"For a capital amount of Rs. {capital}:")
for i in range(N):
    print(f"Invest Rs. {round(w[i][0] * capital, 2)} in {companies[i]}")

print(
    f"\nExpected Return of Entire Portfolio: {round((w.T @ exp_ret)[0][0] * 100, 2)}%"
)
print(f"Risk of Entire Portfolio: {round((w.T @ cov_mat @ w)[0][0] * 100, 2)}%")

For a capital amount of Rs. 100000:
Invest Rs. 18709.1 in HDFCAMC
Invest Rs. 32289.98 in CUMMINSIND
Invest Rs. 21897.27 in MCDOWELL-N
Invest Rs. 10523.1 in LTTS
Invest Rs. 4129.55 in HCLTECH
Invest Rs. 12451.0 in DRREDDY

Expected Return of Entire Portfolio: 27.06%
Risk of Entire Portfolio: 3.21%


In [67]:
pd.DataFrame(cov_mat).to_excel("/tmp/foo.xlsx")

In [69]:
w_eq = np.ones((N, 1)) / N
w_eq.T @ exp_ret
(w_eq.T @ cov_mat @ w).flatten()[0]

0.032034382225316994